Poniżej zamieszczam kod, który umożliwia przewidzenie w wartości T, A, B lub C dla jednego pliku. Pliki znajdują się w folderze datasets/new_Si_jaw_delta.
Wartości T, A, B, C dla poszczególnych plików są zakodowane w ich nazwach -> T_A_B_C. Aby dokonać przewidywania na pliku modelem uczonym na
nieustandaryzowanym zestawie danych należy wywołać funkcję predict. Funkcja przyjmuje jako argumenty zestaw danych (plik txt) oraz model. Wszystkie
modele posiadają 7 wejść oraz 1 wyjście. Wartość X przewidywana przez model, ilość warstw ukrytych w modelu oraz ilość neuronów w każdej warstwie są
zapisane w nazwie modelu w następujący sposób - "modelX_neuronyW1warstwieukrytej_neuronyW2warstwieukrytej_...". Modele uczone na danych standaryzowanych zawierają w sobie frazę "standard". Aby przewidzieć wartość za pomocą modelu
uczonego na danych standaryzowanych należy użyć funkcji predict_scaled. Przyjmuje ona jako argumenty także nazwy scalerów dla feature'ów i targetu.
Przeznaczenie scalera jest opisane w jego nazwie - np. bScaler_featureScaler.pkl jest scalerem użytym dla skalowania feature'ów zastosowanych do uczenia modeli przewidujących wartość B. Funkcję należy wywołać na dole drugiej komórki.


In [15]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, activation_fn=nn.ReLU):

        super(MLP, self).__init__()
        if len(hidden_layers) > 8:
            raise ValueError("The number of hidden layers cannot exceed 8.")

        self.layers = nn.ModuleList()
        prev_size = input_size

        # Create hidden layers
        for neurons in hidden_layers:
            self.layers.append(nn.Linear(prev_size, neurons))
            self.layers.append(activation_fn())  # Add the specified activation function
            prev_size = neurons

        # Create output layer
        self.layers.append(nn.Linear(prev_size, output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [34]:
import os
import joblib
import pandas as pd
import torch
from torch import layout
import statistics


project_folder = os.getcwd()
folder_path = os.path.join(project_folder,"datasets", "new_Si_jaw_delta", "")
all_items = os.listdir(folder_path)
files = [item for item in all_items if os.path.isfile(os.path.join(folder_path, item))]

def extract_from_name(name):
    dataHelper = pd.read_csv(folder_path + name, sep='\t', header=None, index_col=False)
    info = name.split('_')
    T = info[0]
    A = info[1]
    B = info[2]
    C = info[3]
    dataHelper = dataHelper.drop(index=[0])
    dataHelper = dataHelper.drop(columns=[7])
    dataHelper.columns = ['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75']
    dataHelper['T'] = T
    dataHelper['A'] = A
    dataHelper['B'] = B
    C = C.removesuffix(".txt")
    print('x')
    if ("-" in C):
        C = C.removesuffix("e-")
        C = float(C) * 10 ** -5
    elif ("e" in C):
        C = C.removesuffix("e")
        C = float(C) * 10 ** -5

    if (float(C) > 1):
        C = float(C) * 10 ** -5

    dataHelper['C'] = C

    return dataHelper

def decode_model(model):
    model = model.removesuffix(".pth")
    layers = model.split('_')
    layers.pop(0)
    return list(map(int, layers))




def predict(file, model):
    print(file)
    print(model)
    dataHelper = extract_from_name(file)
    x = dataHelper[['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75']]
    x = torch.from_numpy(x.values).float()
    layers = decode_model(model)
    print(layers)
    mModel = MLP(input_size=7, output_size=1, hidden_layers=layers)
    if torch.cuda.is_available():
        mModel.load_state_dict(torch.load(model))
    else:
        mModel.load_state_dict(torch.load(model, map_location=torch.device('cpu')))

    values = []

    for i in x:
        with torch.no_grad():
            values.append(mModel(i).item())

    print(f"Model prediction: {statistics.median(values)}" )



def predict_scaled(file, model, scaler, targetscaler):
    print(file)
    print(model)
    print(scaler)
    print(targetscaler)
    dataHelper = extract_from_name(file)
    x = dataHelper[['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75']]
    x = torch.from_numpy(x.values).float()
    layers = decode_model(model)
    print(layers)
    mModel = MLP(input_size=7, output_size=1, hidden_layers=layers)
    if torch.cuda.is_available():
        mModel.load_state_dict(torch.load(model))
    else:
        mModel.load_state_dict(torch.load(model, map_location=torch.device('cpu')))

    featureNames = ['wavelength', 'psi65', 'del65', 'psi70', 'del70', 'psi75', 'del75']
    featureScaler = joblib.load(scaler)
    targetScaler = joblib.load(targetscaler)
    values = []
    for i in x:
        i_df = pd.DataFrame(i.numpy().reshape(1, -1), columns=featureNames)
        iScaled = featureScaler.transform(i_df)
        with torch.no_grad():
            pred = mModel(torch.from_numpy(iScaled).float())
            numpyPred = pred.detach().cpu().numpy()
            predInv = targetScaler.inverse_transform(numpyPred)
            values.append(predInv.item())

    print(f"Model prediction: {statistics.median(values)}")
    

predict("94.848_1.3902_0.01375279_0.00019072660000000002.txt", "modelT_48_32_16_8.pth")
predict_scaled("94.848_1.3902_0.01375279_0.00019072660000000002.txt", "modelCstandard_256_128_64_32.pth", "cScaler_featureScaler.pkl", "cScaler_targetScaler.pkl")


94.848_1.3902_0.01375279_0.00019072660000000002.txt
modelT_48_32_16_8.pth
x
[48, 32, 16, 8]
Model prediction: 95.28948211669922
94.848_1.3902_0.01375279_0.00019072660000000002.txt
modelCstandard_256_128_64_32.pth
cScaler_featureScaler.pkl
cScaler_targetScaler.pkl
x
[256, 128, 64, 32]
Model prediction: 0.00012547629012260586
